In [154]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import lightgbm as lgb
import catboost as cgb
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score

from skopt import BayesSearchCV
# parameter ranges are specified by one of below

from skopt.space import Real, Categorical, Integer

import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold


In [155]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#### Загрузка выборки

In [156]:
train = pd.read_parquet('data/train.parquet')
train.head()

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,...,_YearBuiltBin_YearBin2,_YearBuiltBin_YearBin3,_YearBuiltBin_YearBin4,_YearBuiltBin_YearBin5,_YearBuiltBin_YearBin6,_YearBuiltBin_YearBin7,_YearRemodAddBin_YearBin4,_YearRemodAddBin_YearBin5,_YearRemodAddBin_YearBin6,_YearRemodAddBin_YearBin7
Id,,,,,,,,,,,,,,,,,,,,,
1,-0.127817,1.209296,0.625446,-0.355892,-0.339062,-0.472456,-0.805990,1.185669,0.548227,0.360672,...,0,0,0,0,0,1,0,0,0,1
2,0.120797,-0.805439,1.257846,-0.355892,0.003303,0.512947,0.433256,-0.867410,-0.378408,-0.054591,...,0,0,0,0,1,0,0,0,1,0
3,0.429834,1.137048,0.113946,-0.355892,0.231214,-0.317122,-0.575842,1.189908,0.680880,0.643806,...,0,0,0,0,0,1,0,0,0,1
4,0.110623,-0.805439,-0.513805,-0.355892,0.348751,-0.715166,-0.436663,1.148647,0.560689,0.804249,...,0,1,0,0,0,0,0,1,0,0
5,0.891805,1.429555,0.506871,-0.355892,0.296484,0.228976,0.122612,1.249303,1.314119,1.719716,...,0,0,0,0,0,1,0,0,0,1


In [157]:
test = pd.read_parquet('data/test.parquet')
test.head()

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,...,_YearBuiltBin_YearBin2,_YearBuiltBin_YearBin3,_YearBuiltBin_YearBin4,_YearBuiltBin_YearBin5,_YearBuiltBin_YearBin6,_YearBuiltBin_YearBin7,_YearRemodAddBin_YearBin4,_YearRemodAddBin_YearBin5,_YearRemodAddBin_YearBin6,_YearRemodAddBin_YearBin7
Id,,,,,,,,,,,,,,,,,,,,,
1461,0.493221,-0.805439,0.072096,2.339152,-0.023846,-0.409352,-0.660217,-0.867410,-1.422865,1.219513,...,0,0,0,1,0,0,0,1,0,0
1462,0.892761,0.983593,1.129971,-0.355892,0.195355,0.675563,0.598430,-0.867410,-0.220639,-0.752989,...,0,0,0,1,0,0,0,1,0,0
1463,0.832144,-0.805439,0.823071,-0.355892,-0.387584,-0.297705,-0.548205,1.125708,0.400198,0.049224,...,0,0,0,0,0,1,0,0,0,1
1464,0.196046,0.355581,0.383646,-0.355892,0.074090,-0.302559,-0.555092,1.115578,0.353021,-0.007402,...,0,0,0,0,0,1,0,0,0,1
1465,-1.148225,-0.805439,-0.404529,-0.355892,0.689482,0.556635,0.478476,-0.867410,-0.335215,0.162478,...,0,0,0,0,0,1,0,0,0,1


In [158]:
test.shape

(1459, 337)

In [159]:
label = pd.read_parquet('data/label.parquet')
label.head()

,SalePrice
Id,
1,208500
2,181500
3,223500
4,140000
5,250000


In [160]:
label_log = label.apply(np.log)

In [161]:
import warnings

warnings.filterwarnings('ignore')

Предсказание будет сделано на основе ансамля моделей 

#### LightGBM Regressor

In [162]:
def bayesion_opt_lgbm(X, y, init_iter=5, n_iters=9, random_state=35, seed = 102, num_iterations = 100):
    dtrain = lgb.Dataset(data=X, label=y)
    def lgb_r2_score(preds, dtrain):
        labels = dtrain.get_label()
        return 'r2', r2_score(labels, preds), True
  # Objective Function
    def hyp_lgbm(num_leaves, feature_fraction, bagging_fraction, max_depth, min_split_gain, min_child_weight):

        params = {'application':'regression','num_iterations': 5000,
                  'learning_rate':0.05, 'early_stopping_round':60,
                  'metric':'l1'} # Default parameters
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, dtrain, nfold=5, seed=35, stratified=False, verbose_eval =None, metrics=['l2'])
        
        return -np.min(cv_result['l2-mean'])
    
    # Domain space-- Range of hyperparameters 
    pds = {'num_leaves': (80, 100),
            'feature_fraction': (0.1, 0.9),
            'bagging_fraction': (0.8, 1),
            'max_depth': (5, 25),
            'min_split_gain': (0.001, 0.1),
            'min_child_weight': (10, 25)
            }

    # Surrogate model
    optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)

    # Optimize
    optimizer.maximize(init_points=init_iter, n_iter=n_iters)
    return optimizer

In [163]:
#Для более качественной оптимизации логарифмирую цену цену
opt = bayesion_opt_lgbm(train, label_log, init_iter=5, n_iters=10, random_state=77, seed = 101, num_iterations = 200)

|   iter    |  target   | baggin... | featur... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        | -0.01476  |  0.9838   |  0.6138   |  20.07    |  12.09    |  0.009645 |  95.76    |
|  2        | -0.01494  |  0.8652   |  0.5329   |  9.805    |  18.18    |  0.04065  |  94.3     |
|  3        | -0.01534  |  0.9673   |  0.5708   |  10.92    |  14.22    |  0.07085  |  88.45    |
|  4        | -0.01548  |  0.8115   |  0.6976   |  14.05    |  12.64    |  0.005888 |  85.85    |
|  5        | -0.01567  |  0.8134   |  0.7009   |  6.275    |  16.48    |  0.03705  |  83.04    |
|  6        | -0.01396  |  0.8003   |  0.2887   |  22.28    |  23.16    |  0.0242   |  99.73    |
|  7        | -0.01474  |  1.0      |  0.1      |  25.0     |  25.0     |  0.1      |  93.65    |
|  8        | -0.01643  |  1.0      |  0.9      |  16.49    |  25.0     |  0.1      |  100.0    |
|  9        | -0.014

In [164]:
params = opt.max["params"]

params['max_depth'] = int(params['max_depth'])
params['num_leaves'] = int(params['num_leaves'])

In [165]:
regr = lgb.LGBMRegressor(**params)

In [166]:
regr.fit(train, label.apply(np.log))
pred_train = regr.predict(train)
pred_test = regr.predict(test)

In [167]:
scor_lgb = pd.DataFrame(index = test.index)
scor_lgb["SalePrice"] = [np.exp(x) for x in pred_test]

#### Catboost Regressor

In [168]:
def cat_hyp(depth, bagging_temperature): # Function to optimize depth and bagging temperature
    
    params = {"iterations": 100,
            "learning_rate": 0.05,
            "loss_function": "RMSE",
            "verbose": False} # Default Parameters
    params[ "depth"] = int(round(depth)) 
    params["bagging_temperature"] = bagging_temperature

    cat_feat = [] # Categorical features list
    cv_dataset = cgb.Pool(data=train,
                  label=label,
                  cat_features=cat_feat)

    scores = cgb.cv(cv_dataset,
              params,
              fold_count=3)
    
    return - np.min(scores['test-RMSE-mean']) 

In [169]:
pds = {'depth': (5, 8),
          'bagging_temperature': (3,10)
          }

In [170]:

# Surrogate model
optimizer = BayesianOptimization(cat_hyp, pds, random_state=2100)
                                  
# Optimize
optimizer.maximize(init_points=3, n_iter=7 )

|   iter    |  target   | baggin... |   depth   |
-------------------------------------------------
|  1        | -2.562e+0 |  8.931    |  5.069    |
|  2        | -2.621e+0 |  3.608    |  6.008    |
|  3        | -2.621e+0 |  6.422    |  5.9      |
|  4        | -2.734e+0 |  7.919    |  7.808    |
|  5        | -2.562e+0 |  4.26     |  5.276    |
|  6        | -2.621e+0 |  5.826    |  5.801    |
|  7        | -2.562e+0 |  10.0     |  5.0      |
|  8        | -2.621e+0 |  10.0     |  6.309    |
|  9        | -2.562e+0 |  7.75     |  5.013    |
|  10       | -2.562e+0 |  3.0      |  5.0      |


In [171]:
optimizer.max

{'target': -25615.237049828265,
 'params': {'bagging_temperature': 8.930629571990341,
  'depth': 5.06944936764366}}

In [172]:
parameters = optimizer.max['params']
parameters['depth'] = int(parameters['depth'])

In [173]:
regr = cgb.CatBoostRegressor(**parameters)

In [174]:
regr.fit(train, label)
pred_train = regr.predict(train)
pred_test = regr.predict(test)

Learning rate set to 0.04173
0:	learn: 74445.3743955	total: 8.87ms	remaining: 8.86s
1:	learn: 72326.6018582	total: 11.4ms	remaining: 5.68s
2:	learn: 70369.6911380	total: 14.2ms	remaining: 4.72s
3:	learn: 68477.2230349	total: 16.9ms	remaining: 4.22s
4:	learn: 66894.7231913	total: 19.4ms	remaining: 3.87s
5:	learn: 65067.6338193	total: 21.9ms	remaining: 3.62s
6:	learn: 63196.5959739	total: 24.7ms	remaining: 3.5s
7:	learn: 61530.1262667	total: 27.3ms	remaining: 3.39s
8:	learn: 60025.4964126	total: 30.1ms	remaining: 3.32s
9:	learn: 58508.0409345	total: 32.6ms	remaining: 3.23s
10:	learn: 57046.3089487	total: 35.3ms	remaining: 3.17s
11:	learn: 55547.9826649	total: 38ms	remaining: 3.13s
12:	learn: 54298.7722450	total: 40.6ms	remaining: 3.08s
13:	learn: 52861.1790154	total: 43.1ms	remaining: 3.03s
14:	learn: 51621.4756092	total: 45.6ms	remaining: 3s
15:	learn: 50479.5035431	total: 48.2ms	remaining: 2.97s
16:	learn: 49313.8314783	total: 51ms	remaining: 2.95s
17:	learn: 48225.6800173	total: 53.6m

161:	learn: 17828.7996450	total: 419ms	remaining: 2.17s
162:	learn: 17810.9431981	total: 437ms	remaining: 2.24s
163:	learn: 17780.9755550	total: 440ms	remaining: 2.24s
164:	learn: 17738.0158847	total: 442ms	remaining: 2.24s
165:	learn: 17702.4282787	total: 445ms	remaining: 2.23s
166:	learn: 17672.7519206	total: 447ms	remaining: 2.23s
167:	learn: 17635.8111869	total: 450ms	remaining: 2.23s
168:	learn: 17609.7879360	total: 452ms	remaining: 2.22s
169:	learn: 17572.9230567	total: 454ms	remaining: 2.22s
170:	learn: 17534.0063626	total: 457ms	remaining: 2.21s
171:	learn: 17501.3738586	total: 459ms	remaining: 2.21s
172:	learn: 17483.3584267	total: 462ms	remaining: 2.21s
173:	learn: 17459.7417547	total: 464ms	remaining: 2.2s
174:	learn: 17425.1903451	total: 466ms	remaining: 2.2s
175:	learn: 17395.4645480	total: 469ms	remaining: 2.19s
176:	learn: 17363.9005620	total: 471ms	remaining: 2.19s
177:	learn: 17330.0710896	total: 474ms	remaining: 2.19s
178:	learn: 17295.5688296	total: 476ms	remaining: 

341:	learn: 14203.1006071	total: 899ms	remaining: 1.73s
342:	learn: 14187.8454769	total: 901ms	remaining: 1.73s
343:	learn: 14165.3752622	total: 904ms	remaining: 1.72s
344:	learn: 14144.2389237	total: 906ms	remaining: 1.72s
345:	learn: 14139.9246785	total: 909ms	remaining: 1.72s
346:	learn: 14115.7856582	total: 912ms	remaining: 1.72s
347:	learn: 14109.2713680	total: 914ms	remaining: 1.71s
348:	learn: 14106.8373817	total: 917ms	remaining: 1.71s
349:	learn: 14084.4098051	total: 919ms	remaining: 1.71s
350:	learn: 14075.9751440	total: 922ms	remaining: 1.7s
351:	learn: 14057.5286611	total: 924ms	remaining: 1.7s
352:	learn: 14038.2169315	total: 927ms	remaining: 1.7s
353:	learn: 14021.3930375	total: 930ms	remaining: 1.7s
354:	learn: 13996.3990131	total: 932ms	remaining: 1.69s
355:	learn: 13973.5647449	total: 935ms	remaining: 1.69s
356:	learn: 13971.4849979	total: 937ms	remaining: 1.69s
357:	learn: 13963.5625510	total: 940ms	remaining: 1.68s
358:	learn: 13946.6048475	total: 943ms	remaining: 1.

520:	learn: 11822.7354995	total: 1.44s	remaining: 1.32s
521:	learn: 11821.4475359	total: 1.44s	remaining: 1.32s
522:	learn: 11814.4225456	total: 1.45s	remaining: 1.32s
523:	learn: 11802.0267125	total: 1.45s	remaining: 1.31s
524:	learn: 11788.9553600	total: 1.45s	remaining: 1.31s
525:	learn: 11783.9770487	total: 1.45s	remaining: 1.31s
526:	learn: 11778.3382234	total: 1.46s	remaining: 1.31s
527:	learn: 11766.6964317	total: 1.46s	remaining: 1.3s
528:	learn: 11757.3218326	total: 1.46s	remaining: 1.3s
529:	learn: 11746.9439237	total: 1.46s	remaining: 1.3s
530:	learn: 11729.8295900	total: 1.47s	remaining: 1.29s
531:	learn: 11724.9419401	total: 1.47s	remaining: 1.29s
532:	learn: 11721.8328165	total: 1.47s	remaining: 1.29s
533:	learn: 11712.8439156	total: 1.48s	remaining: 1.29s
534:	learn: 11695.5916219	total: 1.48s	remaining: 1.28s
535:	learn: 11683.4131137	total: 1.48s	remaining: 1.28s
536:	learn: 11676.4439384	total: 1.49s	remaining: 1.28s
537:	learn: 11674.4195767	total: 1.49s	remaining: 1

673:	learn: 10256.5207254	total: 1.98s	remaining: 958ms
674:	learn: 10249.0043621	total: 1.98s	remaining: 955ms
675:	learn: 10239.3933687	total: 1.99s	remaining: 954ms
676:	learn: 10238.1218847	total: 1.99s	remaining: 951ms
677:	learn: 10229.2412516	total: 2s	remaining: 948ms
678:	learn: 10213.2628486	total: 2s	remaining: 945ms
679:	learn: 10207.2782092	total: 2s	remaining: 942ms
680:	learn: 10195.3852975	total: 2s	remaining: 939ms
681:	learn: 10183.1270642	total: 2.01s	remaining: 937ms
682:	learn: 10175.1282361	total: 2.01s	remaining: 934ms
683:	learn: 10162.8523176	total: 2.02s	remaining: 931ms
684:	learn: 10159.1077571	total: 2.02s	remaining: 929ms
685:	learn: 10158.0530141	total: 2.02s	remaining: 926ms
686:	learn: 10149.1667847	total: 2.03s	remaining: 924ms
687:	learn: 10136.7523284	total: 2.03s	remaining: 921ms
688:	learn: 10120.4540741	total: 2.03s	remaining: 918ms
689:	learn: 10113.9826902	total: 2.04s	remaining: 915ms
690:	learn: 10106.0029983	total: 2.04s	remaining: 912ms
691:

842:	learn: 8906.8683150	total: 2.6s	remaining: 485ms
843:	learn: 8901.7699455	total: 2.61s	remaining: 482ms
844:	learn: 8889.1784485	total: 2.61s	remaining: 479ms
845:	learn: 8880.3941816	total: 2.61s	remaining: 476ms
846:	learn: 8869.7089837	total: 2.62s	remaining: 473ms
847:	learn: 8862.5995009	total: 2.62s	remaining: 469ms
848:	learn: 8855.2545808	total: 2.62s	remaining: 466ms
849:	learn: 8849.4402495	total: 2.62s	remaining: 463ms
850:	learn: 8842.7825537	total: 2.63s	remaining: 460ms
851:	learn: 8838.5611190	total: 2.63s	remaining: 457ms
852:	learn: 8832.5551178	total: 2.63s	remaining: 454ms
853:	learn: 8820.4106093	total: 2.63s	remaining: 450ms
854:	learn: 8816.6877152	total: 2.64s	remaining: 447ms
855:	learn: 8804.0193391	total: 2.64s	remaining: 444ms
856:	learn: 8797.2957977	total: 2.64s	remaining: 441ms
857:	learn: 8783.9334322	total: 2.64s	remaining: 438ms
858:	learn: 8776.7691349	total: 2.65s	remaining: 434ms
859:	learn: 8767.2303571	total: 2.65s	remaining: 431ms
860:	learn:

In [175]:
scor_cgb = pd.DataFrame(index = test.index)
scor_cgb["SalePrice"] = pred_test

CatBoosting Лучше себя показывает если цель не логарифмировать

#### XGBRegressor

Как ни странно именно коэффицент детерминации позволяет подогнать гипермпараметры наилучшим образом

In [176]:

# Define R2
def xgb_r2(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(preds, labels)
  
dtrain = xgb.DMatrix(train, label, feature_names=train.columns.values)

# Define Objective Function
def hyp_xgb(max_depth, subsample, colsample_bytree,min_child_weight, gamma ):
    params = {
    'n_estimators': 300,
    'eta': 0.05,
    'objective': 'reg:linear',
    'eval_metric':'mae', # Optional --> Use eval_metric if you want to stop evaluation based on eval_metric 
    'silent': 1
     }
    params['max_depth'] = int(round(max_depth))
    params['subsample'] = max(min(subsample, 1), 0)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['min_child_weight'] = int(min_child_weight)
    params['gamma'] = max(gamma, 0)
    scores = xgb.cv(params, dtrain, num_boost_round=1000,verbose_eval=False, early_stopping_rounds=10, feval=xgb_r2, maximize=True, nfold=5)
    return  -scores['test-mae-mean'].iloc[-1]

In [177]:
pds ={
  'min_child_weight':(14, 20),
  'gamma':(0, 5),
  'subsample':(0.5, 1),
  'colsample_bytree':(0.1, 1),
  'max_depth': (5, 10)
}

In [178]:

optimizer = BayesianOptimization(hyp_xgb, pds, random_state=10)
                                  
# Optimize

optimizer.maximize(init_points=5, n_iter=20)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------
[16:02:32] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:02:32] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:02:32] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:02:32] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language

|  4        | -1.412e+0 |  0.6513   |  3.609    |  6.459    |  19.51    |  0.8573   |
[16:02:45] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:02:45] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:02:45] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:02:45] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  8        | -1.389e+0 |  0.5169   |  0.6267   |  8.375    |  14.95    |  0.5867   |
[16:02:58] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:02:58] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:02:58] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:02:58] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  12       | -1.423e+0 |  0.9743   |  0.8065   |  6.323    |  19.21    |  0.8489   |
[16:03:11] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:11] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:11] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:11] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  16       | -1.408e+0 |  0.5717   |  4.339    |  7.736    |  18.62    |  0.7802   |
[16:03:23] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:23] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:23] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:23] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  20       | -1.407e+0 |  0.7942   |  2.233    |  7.499    |  16.94    |  0.8463   |
[16:03:34] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:34] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:34] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:34] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

|  24       | -1.422e+0 |  0.7633   |  4.751    |  8.943    |  19.75    |  0.6639   |
[16:03:46] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:46] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:46] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[16:03:46] WARNING: ..\src\learner.cc:541: 
Parameters: { n_estimators, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip

In [179]:
optimizer.max

{'target': -13817.745312399999,
 'params': {'colsample_bytree': 0.5831164523882694,
  'gamma': 1.9830124103987823,
  'max_depth': 5.18956303609651,
  'min_child_weight': 15.301372552109147,
  'subsample': 0.6140849152343821}}

In [180]:
parameters =  optimizer.max['params']


parameters['max_depth'] = int(parameters['max_depth'])

In [181]:
regr = xgb.XGBRegressor(**parameters)

In [182]:
regr.fit(train, label)
pred_train = regr.predict(train)
pred_test = regr.predict(test)

In [183]:
scor_xgb = pd.DataFrame(index = test.index)
scor_xgb["SalePrice"] = pred_test

In [184]:
scor_xgb.to_csv("xgb.csv")

#### Random Forest Regressor

In [185]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [186]:
parameters = {
   'n_estimators': [100, 200, 400],
   'max_features': ['auto', 'sqrt', 'log2'],
   'max_depth' : [4,8,16,24,32],
   'criterion' : ['mse', 'mae'],
    'min_impurity_decrease' : [0.0, 1e-6, 1e-5, 1e-4, 1e-3]
}


In [187]:
grid = RandomizedSearchCV(
    RandomForestRegressor(),
    parameters,
    n_iter = 150,
    n_jobs = -1,
    cv = 3,
    scoring =  "r2",
    verbose = 1
)

In [188]:
#grid.fit(train, label)

In [189]:
#grid.best_score_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_score_'

In [190]:
regr = RandomForestRegressor(max_depth = 24, min_impurity_decrease = 1e-5)

In [191]:
regr.fit(train, label)
pred_train = regr.predict(train)
pred_test = regr.predict(test)

In [192]:
scor_rf = pd.DataFrame(index = test.index)
scor_rf["SalePrice"] = pred_test

In [193]:
scor_rf.to_csv('rf.csv')

#### Submission

Чтобы уменьшить влияние возможных выбросов используем логарифм

In [194]:
seria = (scor_lgb["SalePrice"].apply(np.log) * 2 +\
         scor_cgb["SalePrice"].apply(np.log) +\
         scor_xgb["SalePrice"].apply(np.log) +\
         scor_rf["SalePrice"].apply(np.log)) / 5

In [195]:
seria = seria.apply(np.exp)

In [196]:
scor_all = pd.DataFrame(index = test.index)
scor_all["SalePrice"] = seria

In [197]:
scor_all.to_csv("submission.csv")

In [ ]:
from sklearn.ensemble